<a href="https://colab.research.google.com/github/Ishaqgopang/Assignment/blob/main/assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
!pip install --upgrade --quiet google-genai

In [84]:
!pip install gTTS --quiet

In [85]:
!pip install playsound

In [93]:
from google.colab import userdata
api_key:str = userdata.get('GOOGLE_API_KEY_3')
if api_key:
  print("api key found")
else:
  print("api key not found")

api key found


In [94]:
from google import genai
from google.genai import Client
client: Client = genai.Client(
    api_key = api_key,

)
model: str = "gemini-2.0-flash-exp"

In [95]:
video_file_name = "/content/Introductory.mp4"

In [96]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROSSING":
    print("Waiting For Video To Be Processed")
    time.sleep(60)
    video_file = client.files.get(name=video_file.name or "")
  if video_file.state == "FAILED ":
    raise ValueError(video_file.state)
  print(f"Video Processing Complete: " + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Video Processing Complete: https://generativelanguage.googleapis.com/v1beta/files/lt9iv4t47ez


In [97]:
from google.genai.types import Content, Part

prompt = """
For each scene in this video,
generate captions that describe the scene along with any spoken text placed in qutation marks.
Place each caption into an object with the timecode of the caption in the video
"""

video = my_video
response = client.models.generate_content(
    model = model,
    contents=[
        Content(
            role ="USER",
            parts=[
                Part.from_uri(
                    file_uri = video.uri or "",
                    mime_type = video.mime_type or ""
                ),
            ]
        ),
        prompt,
    ]
)
print(response.text)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': "Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:602140094603'.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_location': 'us-central2', 'quota_limit_value': '0', 'service': 'generativelanguage.googleapis.com', 'quota_metric': 'generativelanguage.googleapis.com/generate_content_requests', 'consumer': 'projects/602140094603', 'quota_limit': 'GenerateContentRequestsPerMinutePerProjectPerRegion'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]}}